# Clasificación de reviews negativas y detección de tópicos

## **1. Clasificación de Reviews**

In [1]:
# Importar librerías
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Descargar recursos necesarios de NLTK
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcbar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jcbar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Unir columnas "*Summary*" y "*Text*" en una sola: "*Review*"

In [ ]:
# Cargar los datos de entrenamiento y prueba
train_path = "Data/final-train.csv"
test_path = "Data/test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Unir las columnas "Summary" y "Text" en una sola columna llamada "Review"
train_df['Review'] = train_df['Summary'].fillna('') + " " + train_df['Text'].fillna('')
test_df['Review'] = test_df['Summary'].fillna('') + " " + test_df['Text'].fillna('')

# Eliminar las columnas originales "Summary" y "Text"
train_df = train_df.drop(columns=['Summary', 'Text'])
test_df = test_df.drop(columns=['Summary', 'Text'])

# Guardar los resultados en archivos CSV actualizados para los siguientes pasos
train_df.to_csv("Data/train-processed.csv", index=False)
test_df.to_csv("Data/test-processed.csv", index=False)


In [3]:
# Imprimir las primeras filas para verificar el resultado
print("Datos de entrenamiento procesados:")
print(train_df.head())

print("\nDatos de prueba procesados:")
print(test_df.head())

Datos de entrenamiento procesados:
   Score                                             Review
0      5  Can really notice when I am not drinking it I ...
1      4  Okay, just not the best This is a decent balsa...
2      3  goo source of fiber, not so much a source of c...
3      5  Great GF Staple I have a 4 yr old and a 2 yr o...
4      5  Coffee just doesn't taste right without Sweet ...

Datos de prueba procesados:
                                              Review
0  I love Hills! My cat is picky, especially when...
1  Thank you, Amazon! My mom, who has always beli...
2  Good product, good price This is my third cont...
3  Soooo Good! The first time I had a cup of this...
4  Not as good as Libby's; cans in bad condition ...


### **Para Random Forest:** 
### Preprocesamiento de datos: eliminación de stopwords, caracteres especiales, conversión a minúsculas, etc.

In [ ]:
# Obtener stopwords en inglés
stop_words = set(stopwords.words('english'))

# Función para limpiar el texto
def clean_text_with_nltk(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales, números y puntuación
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenizar el texto
    words = nltk.word_tokenize(text)
    # Eliminar stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Aplicar la limpieza a la columna 'Review' en los datos de entrenamiento y prueba
train_df['Cleaned_Review'] = train_df['Review'].apply(clean_text_with_nltk)
test_df['Cleaned_Review'] = test_df['Review'].apply(clean_text_with_nltk)

# Guardar los datos con la nueva columna 'Cleaned_Review'
train_df.to_csv("Data/train-cleaned.csv", index=False)
test_df.to_csv("Data/test-cleaned.csv", index=False)



In [5]:
# Imprimir las primeras filas para verificar el resultado
print("Datos de entrenamiento con texto limpio (NLTK):")
print(train_df[['Review', 'Cleaned_Review']].head())

print("\nDatos de prueba con texto limpio (NLTK):")
print(test_df[['Review', 'Cleaned_Review']].head())

Datos de entrenamiento con texto limpio (NLTK):
                                              Review  \
0  Can really notice when I am not drinking it I ...   
1  Okay, just not the best This is a decent balsa...   
2  goo source of fiber, not so much a source of c...   
3  Great GF Staple I have a 4 yr old and a 2 yr o...   
4  Coffee just doesn't taste right without Sweet ...   

                                      Cleaned_Review  
0  really notice drinking hormone problems past c...  
1  okay best decent balsamic glazereducationbr br...  
2  goo source fiber much source chocoate cookie m...  
3  great gf staple yr old yr old cant eat gluten ...  
4  coffee doesnt taste right without sweet n low ...  

Datos de prueba con texto limpio (NLTK):
                                              Review  \
0  I love Hills! My cat is picky, especially when...   
1  Thank you, Amazon! My mom, who has always beli...   
2  Good product, good price This is my third cont...   
3  Soooo Good! The 

### Aplicar TFIDF

Corroborar que no se hayan generados valores nulos

In [17]:
# Contar valores nulos en la columna Cleaned_Review
train_null_count = train_df['Cleaned_Review'].isnull().sum()
test_null_count = test_df['Cleaned_Review'].isnull().sum()

# Contar valores vacíos (incluyendo espacios en blanco)
train_empty_count = (train_df['Cleaned_Review'].fillna('').str.strip() == '').sum()
test_empty_count = (test_df['Cleaned_Review'].fillna('').str.strip() == '').sum()

# Imprimir resultados
print(f"Valores nulos en entrenamiento: {train_null_count}")
print(f"Valores vacíos en entrenamiento (incluidos los nulos): {train_empty_count}")

print(f"\nValores nulos en prueba: {test_null_count}")
print(f"Valores vacíos en prueba (incluidos los nulos): {test_empty_count}")

Valores nulos en entrenamiento: 1
Valores vacíos en entrenamiento (incluidos los nulos): 1

Valores nulos en prueba: 0
Valores vacíos en prueba (incluidos los nulos): 0


Eliminar valores nulos

In [18]:
# Verificar y manejar valores nulos en la columna Cleaned_Review
train_df['Cleaned_Review'] = train_df['Cleaned_Review'].fillna('')
test_df['Cleaned_Review'] = test_df['Cleaned_Review'].fillna('')

# Eliminar registros que puedan estar vacíos tras el preprocesamiento
train_df = train_df[train_df['Cleaned_Review'].str.strip() != '']
test_df = test_df[test_df['Cleaned_Review'].str.strip() != '']

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializar el vectorizador TF-IDF
tfidf = TfidfVectorizer(
    max_features=10000,  # Limitar a 10,000 términos más frecuentes
    stop_words='english',  # Eliminar stopwords adicionales si quedaron
    ngram_range=(1, 2),  # Incluir unigrams y bigrams
    max_df=0.9,  # Excluir términos demasiado frecuentes
    min_df=5  # Excluir términos muy raros
)

# Ajustar el vectorizador al conjunto de entrenamiento y transformar
X_train_tfidf = tfidf.fit_transform(train_df['Cleaned_Review'])
X_test_tfidf = tfidf.transform(test_df['Cleaned_Review'])

# Guardar el mapeo entre palabras y sus índices para referencia
feature_names = tfidf.get_feature_names_out()


In [20]:
# Imprimir el tamaño de los datos transformados
print(f"Datos de entrenamiento TF-IDF shape: {X_train_tfidf.shape}")
print(f"Datos de prueba TF-IDF shape: {X_test_tfidf.shape}")

Datos de entrenamiento TF-IDF shape: (426339, 10000)
Datos de prueba TF-IDF shape: (7105, 10000)


### **Para modelo tipo BERT:** 
### No se necesita de un preprocesamiento adicional, solo tokenización con el tokenizador propio de BERT. 

Dado el lenguaje informal observado en los reviews, probaremos primero con el modelo *bertweet-base*. Este fue entrenado específicamente con datos de twitter y se desempeña bien en datasets con textos informales. 

In [21]:
from transformers import AutoTokenizer

# Cargar el tokenizador para bertweet-base
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=True)

# Tokenizar los datos de entrenamiento
train_tokens = tokenizer(
    train_df['Review'].tolist(),  # Lista de textos a tokenizar
    padding=True,                # Agregar padding a la longitud máxima
    truncation=True,             # Truncar textos largos al límite máximo del modelo
    max_length=128,              # Longitud máxima de los textos tokenizados
    return_tensors="pt"          # Regresar tensores de PyTorch
)

# Tokenizar los datos de prueba
test_tokens = tokenizer(
    test_df['Review'].tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

c:\Users\jcbar\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jcbar\.cache\huggingface\hub\models--vinai--bertweet-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [25]:
# Imprimir algunos ejemplos para verificar
print("Ejemplo de tokens del conjunto de entrenamiento:\n")
print(train_tokens['input_ids'][0])  # IDs de tokens
print("\nEjemplo de tokens del conjunto de prueba:\n")
print(test_tokens['input_ids'][0])

Ejemplo de tokens del conjunto de entrenamiento:

tensor([    0,   427,   116,  1607,    64,     8,   155,    46,  2065,    18,
            8,    36,   118, 49232,  1191,    16,     6,   648,   992,    15,
          254,    13,   177,   235,   106,  6719,    13,  8785,    59,    41,
          154,    50,    15,  5168,     8,   436,   357,    19,  2642,  2860,
            9, 27786,   639, 10584, 18871,  2603, 27067, 10584, 18871,  2603,
        27067,   126,    17,    63,    15,    23, 17786,    61,  6580,     4,
            8,    36,   108,  2065,    33,    19,   163, 29187,    77,     8,
         1555,   297,   119,   825,    50,    30,    11,  2096,    15,    93,
           13,   167,   223,    50,  3749,    18,     9, 12292, 10130,    11,
         4069,     4,     8,    56,  1607,    64,     8, 58297, 15700,   108,
         2065,   987,     7,    23,  4741,   297,     9,    51,    50,    15,
         1205,    13,     8,   174,   249,  5647,  2154,   639, 10584, 18871,
         2603,

### Fine tuning de Bertweet

#### Entrenamiento del modelo pre-entrenado BERT utilizando el dataset etiquetado (calificaciones 1-5).

#### Validación para ajustar hiperparámetros (learning rate, batch size, etc.).

### Entrenamiento y validación de Random Forest.

### Evaluación con conjunto de prueba (Recall, F1-score y Matriz de Confusión).

### Generación de CSVs con etiquetas predichas: 1 (negativo; calificaciones 1 y 2) y 0 (no negativo; calificaciones 3-5).

## **2. Análisis de Tópicos:**

### Del dataset de entrenamiento, extraer opiniones negativas (calificaciones 1 y 2).

### Generar embeddings (usar modelo BERT al que le hayamos hecho fine tuning).

### Determinar la cantidad de clusters ideales utilizando silhouette_score y el criterio del codo.

### Aplicar K-Means con el número óptimo de clusters.

### Mostrar tópicos con ejemplos representativos (e.g., textos más cercanos al centroide de cada cluster).

### Crear nubes de palabras para cada cluster.